In [ ]:
import os
import cv2
import glob
import ntpath
import numpy
import pandas

In [ ]:
src_path = "G:/LungCancerPredict/detected/cnn_detect_ndsb3/"
dst_path = "G:/LungCancerPredict/detected/cnn_detect_ndsb3_nms/"
os.makedirs(dst_path, exist_ok=True)

for index, csv_file in enumerate(glob.glob(src_path + "*.csv")):

    df = pandas.read_csv(csv_file)

    patient_id = ntpath.basename(csv_file).replace(".csv", "") 
    
    csv_target_path = dst_path + patient_id + ".csv"
    
    patient_img = load_patient_images(patient_id)

    if len(df）>0:

        coord = numpy.array( df.loc[:,['coord_x','coord_y','coord_z'] ] )
        confidence = numpy.array( df.loc[:,['nodule_chance']] )[:,0]
      
        AB = toAB(coord,patient_img.shape)

        keep = NMS(AB,confidence,0.2)

        print(df.iloc[keep])

        df = pandas.DataFrame(df.iloc[keep], columns=["anno_index", "coord_x", "coord_y", "coord_z", "nodule_chance"])
        df.to_csv(csv_target_path, index=False)


In [ ]:
def load_patient_images(patient_id):

    src_path = "G:/LungCancerPredict/extracted/ndsb3_extracted_images/" + patient_id + "/"
    src_files = glob.glob(src_path + "*_i.png")
    
    src_files.sort()
    
    imgs = [cv2.imread(src_file, cv2.IMREAD_GRAYSCALE) for src_file in src_files]
    imgs = [img.reshape((1, ) + img.shape) for img in imgs]
    
    res = numpy.vstack(imgs)
    
    return res

#由中心点的坐标，求bbox的A和B 
def toAB(coord,shape):
    
    AB=[]; CUBE_SIZE = 32
    
    for i in range(coord.shape[0]):
        
        temp=[]
        
        centr_X=int(coord[i][0] * shape[1])
        centr_Y=int(coord[i][1] * shape[2])
        centr_Z=int(coord[i][2] * shape[0])
        
        A=[ centr_X-CUBE_SIZE/2 , centr_Y-CUBE_SIZE/2 , centr_Z-CUBE_SIZE/2 ]  #A永远是坐标值较小的那边
        B=[ centr_X+CUBE_SIZE/2 , centr_Y+CUBE_SIZE/2 , centr_Z+CUBE_SIZE/2 ]
        
        temp=[]
        temp.append(A)
        temp.append(B)
    
        AB.append(temp)
        
    return numpy.array(AB)

def NMS(data,confidence,threshold):      
    
    A=data[:,0,:]
    B=data[:,1,:]
   
    scores = confidence  #bbox打分      

    areas = (B[:,0] - A[:,0] + 1) * (B[:,1] - A[:,1] + 1) * (B[:,2] - A[:,2] + 1)   
    
    #打分从大到小排列，取index      
    order = scores.argsort()[::-1]  

    #keep为最后保留的边框      
    keep = []     
    while order.size > 0: 
        
        #order[0]是当前分数最大的窗口，肯定保留          
        i = order[0]         
        keep.append(i)  
        
        #计算窗口i与其他所有窗口的交叠部分的面积  
        A_X = numpy.maximum(A[i,0], A[order[1:],0])  
        A_Y = numpy.maximum(A[i,1], A[order[1:],1])  
        A_Z = numpy.maximum(A[i,2], A[order[1:],2])  
        
        B_X = numpy.minimum(B[i,0], B[order[1:],0])  
        B_Y = numpy.minimum(B[i,1], B[order[1:],1])  
        B_Z = numpy.minimum(B[i,2], B[order[1:],2])  
          
        x = numpy.maximum(0.0, B_X - A_X +1 )   
        
        y = numpy.maximum(0.0, B_Y - A_Y +1 )     

        z = numpy.maximum(0.0, B_Z - A_Z +1 )  
              
        inter = x * y * z  #交/并得到iou值 
          
        IoU = inter / (areas[i] + areas[order[1:]] - inter) 

        #index为所有与窗口i的iou值小于threshold值的窗口的index，其他窗口此次都被窗口i吸收      
        index = numpy.where(IoU <= threshold)[0]  
        
        #order里面只保留与窗口i交叠面积小于threshold的那些窗口，由于IoU长度比order长度少1(不包含i)，所以index+1对应到保留的窗口     
        order = order[index + 1]          
    
    return keep